In [ ]:
import os
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import time

# Inisialisasi klien
client = OpenAI(api_key="YOUR_API_KEY")  # Ganti dengan API key-mu

# Daftarkan tqdm dengan pandas untuk progress bar yang cantik
tqdm.pandas(desc="Membersihkan Berita")

# Aturan pembersih teks (bagian "cached" / system prompt)
CLEANING_RULES = '''
Anda adalah asisten pembersih teks berita. Tugas Anda adalah membersihkan konten berita dari elemen non-esensial dengan aturan berikut:

1. HAPUS seluruh teks "Advertisement", "ADVERTISEMENT", atau varian iklan lainnya.

2. HAPUS bagian "Baca Juga", "Simak Juga", dan referensi artikel terkait beserta tautannya.

3. HAPUS disclaimer (contoh: "Disclaimer: Setiap keputusan investasi..."), termasuk teks setelahnya hingga akhir kalimat.

4. HAPUS informasi penulis/sumber di akhir teks (contoh: "Penulis: Fitri Rahayu Amanda", "Sumber: Huffington Post").

5. HAPUS teks promosi/platform eksternal (contoh: "Dikutip dari YouTube Fesyen Triki", "Download aplikasi Vidio sekarang").

6. HAPUS tag media sosial (contoh: "[Gambas:Video 20detik]", "[Gambas:Instagram]").

7. HAPUS header berulang (contoh: "Liputan6.com, Jakarta" yang muncul berkali-kali).

8. HAPUS kalimat ajakan interaktif (contoh: "Bagikan artikel ini", "Beri komentar").

9. PERTAHANKAN:
   - Inti berita (fakta, peristiwa, data)
   - Kutipan langsung narasumber
   - Data statistik/referensi resmi

10. NORMALISASI:
    - Ganti tanda petik kotor (""...""") dengan tanda petik standar ("...")
    - Perbaiki spasi tidak konsisten (contoh: "kesehatan mentalyang" → "kesehatan mental yang")

Berikan HANYA teks yang sudah dibersihkan tanpa komentar tambahan.
'''

def generate_clean_prompt(isi_berita: str) -> list:
    """
    Membuat daftar pesan ChatCompletion (system + user)
    """
    return [
        {"role": "system", "content": CLEANING_RULES},
        {"role": "user", "content": f'Berikut teks yang perlu dibersihkan:\n"""\n{isi_berita}\n"""'}
    ]

def clean_berita(isi_berita: str) -> str:
    """
    Mengirim permintaan ke GPT-4o-mini untuk membersihkan teks
    """
    messages = generate_clean_prompt(isi_berita)
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content.strip()

# --------------------
# Contoh penggunaan
# --------------------
if __name__ == "__main__":
    # masukkan dataset-
    df = pd.read_csv('scrapping_news_18-20_fix.csv')

    # 2. ITERASI & PROSES PEMBERSIHAN MENGGUNAKAN .progress_apply()
    # .progress_apply akan menampilkan progress bar berkat `tqdm.pandas()`
    # Hasilnya akan disimpan di kolom baru bernama 'berita_bersih'
    print("\nMemulai proses pembersihan berita...")
    df['berita_bersih'] = df['isi_berita'].progress_apply(clean_berita)
    
    # 3. TAMPILKAN DAN SIMPAN HASIL
    print("\nProses pembersihan selesai. DataFrame hasil:")
    print(df[['judul', 'berita_bersih']])
    
    # Simpan hasilnya ke file CSV baru
    output_filename = 'hasil_berita_bersih.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8')
    print(f"\nHasil telah disimpan ke dalam file: {output_filename}")



Memulai proses pembersihan berita...


Membersihkan Berita: 100%|██████████| 20/20 [04:30<00:00, 13.51s/it]



Proses pembersihan selesai. DataFrame hasil:


KeyError: "['judul_berita'] not in index"

In [14]:
df.head()

,sumber,url,judul,author,tanggal_publikasi,isi_berita,berita_bersih
0,okezone.com,https://techno.okezone.com/read/2025/06/16/57/...,Tak Perlu Panik Jika HP Android Hilang Atau Di...,Fitri Rahayu Amanda,", Jurnalis-Senin, 16 Juni 2025 |15:20 WIB",JAKARTA– Kehilangan ponsel atau HP adalah keja...,Kehilangan ponsel atau HP adalah kejadian yang...
1,liputan6.com,https://www.liputan6.com/citizen6/read/6055023...,5 Kebiasaan Kerja Seperti Ini Diam-Diam Ternya...,Sulung Lahitani,"Diperbarui 18 Jun 2025, 14:05 WIB","Liputan6.com, JakartaKita semua mengalami pasa...",Kita semua mengalami pasang surut di tempat ke...
2,liputan6.com,https://www.liputan6.com/global/read/6055796/k...,Kondisi Terkini Ayatollah Khamenei Dikabarkan ...,Tanti Yulianingsih,"Diperbarui 18 Jun 2025, 17:07 WIB","Liputan6.com, Teheran -Setelah jadi sorotan ka...",Teheran - Setelah jadi sorotan karena jadi tar...
3,liputan6.com,https://www.liputan6.com/islami/read/6054124/1...,11 Foto Ide Padu Padan Layering Outfit ala Ric...,Muhamad Husni Tamami,"Diperbarui 17 Jun 2025, 04:30 WIB","Liputan6.com, Jakarta -Layering outfitadalah s...",Layering outfit adalah salah satu trik padu-pa...
4,liputan6.com,https://www.liputan6.com/crypto/read/6051205/h...,Harga Kripto Hari Ini 13 Juni 2025: Bitcoin da...,Natasha Amani,"Diperbarui 13 Jun 2025, 08:14 WIB","Liputan6.com, Jakarta -HargaBitcoindan kripto ...",Harga Bitcoin dan kripto teratas lainnya menga...
